In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import numpy as np
import qiime2 as q2
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
!pwd

/home/jovyan/Group_project


In [2]:
data_dir = 'cancer_data'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

In [5]:
! wget -nv -O $data_dir/sequence.qza 'https://polybox.ethz.ch/index.php/s/PCQspFMocVCKjZ3/download'

2022-10-14 10:07:15 URL:https://polybox.ethz.ch/index.php/s/PCQspFMocVCKjZ3/download [3433846903/3433846903] -> "cancer_data/sequence.qza" [1]


In [ ]:
#! wget -nv -O $data_dir/meta_data.tsv 'https://polybox.ethz.ch/index.php/s/r1AYzdUVWnQyiRL/download'

In [7]:
!qiime tools peek $data_dir/sequence.qza

UUID:        394c4773-80e2-46a6-9fba-40e7c8ec3fb9
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


In [8]:
! qiime demux summarize \
    --i-data $data_dir/sequence.qza \
    --o-visualization $data_dir/sequence.qzv

Saved Visualization to: cancer_data/sequence.qzv


In [6]:
Visualization.load(f'{data_dir}/sequence.qzv')

<visualization: Visualization uuid: 613ea6f5-21e8-46fc-b274-9159be0df6eb>

In [14]:
meta_df = pd.read_csv("/home/jovyan/Group_project/cancer_data/meta_data.tsv", sep='\t',index_col=0)
meta_df.head(n=5)

,stool_consistency,hct_source,disease,categorical_time_relative_to_engraftment,week_relative_to_hct,timepoint_of_transplant,day_relative_to_nearest_hct,alleged_abduction
sampleid,,,,,,,,
N4VICF,formed,cord,Myelodysplastic Syndromes,pre,-1.0,6.0,-6.0,0
8A0F9A,formed,cord,Leukemia,pre,-2.0,7.0,-7.0,0
5Y49IM,semi-formed,cord,Leukemia,peri,-1.0,7.0,0.0,1
ZKJI45,semi-formed,cord,Leukemia,post,1.0,7.0,8.0,0
2I7SIQ,liquid,cord,Leukemia,peri,-1.0,0.0,0.0,1


In [12]:
meta_df.columns

Index(['stool_consistency', 'hct_source', 'disease',
       'categorical_time_relative_to_engraftment', 'week_relative_to_hct',
       'timepoint_of_transplant', 'day_relative_to_nearest_hct',
       'alleged_abduction'],
      dtype='object')

In [13]:
meta_df.groupby(['alleged_abduction']).describe()

week_relative_to_hct                                     \
                                 count      mean       std  min  25%  50%   
alleged_abduction                                                           
0                                131.0 -0.312977  1.030937 -2.0 -1.0 -1.0   
1                                 26.0  0.384615  1.061204 -1.0 -1.0  1.0   

                            timepoint_of_transplant             ...       \
                   75%  max                   count       mean  ...  75%   
alleged_abduction                                               ...        
0                  1.0  2.0                   131.0  14.007634  ...  6.0   
1                  1.0  2.0                    26.0  37.500000  ...  8.5   

                         day_relative_to_nearest_hct                           \
                     max                       count      mean       std  min   
alleged_abduction                                                               
0                  379.0                       131.0  1.885496  7.283956 -9.0   
1                  379.0                        26.0  7.153846  6.943730 -4.0   

                                          
                   25%   50%   75%   max  
alleged_abduction                         
0                 -5.0   0.0   8.0  16.0  
1                  0.0  10.0  12.0  16.0  

[2 rows x 24 columns]

how dereplicated? 下面

In [6]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_dir/sequence.qza \
    --p-trunc-len-f 150 \
    --p-trunc-len-r 157 \
    --p-n-threads 4 \
    --o-table $data_dir/dada2_table.qza \
    --o-representative-sequences $data_dir/dada2_rep_set.qza \
    --o-denoising-stats $data_dir/dada2_stats.qza

Saved FeatureTable[Frequency] to: cancer_data/dada2_table.qza
Saved FeatureData[Sequence] to: cancer_data/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: cancer_data/dada2_stats.qza


In [15]:
! qiime feature-table summarize \
  --i-table $data_dir/dada2_table.qza \
  --o-visualization $data_dir/dada2_table.qzv 

Saved Visualization to: cancer_data/dada2_table.qzv


In [12]:
! qiime feature-table tabulate-seqs \
  --i-data $data_dir/dada2_rep_set.qza \
  --o-visualization $data_dir/dada2_rep_set.qzv

Saved Visualization to: cancer_data/dada2_rep_set.qzv


In [13]:
! qiime metadata tabulate \
  --m-input-file $data_dir/dada2_stats.qza \
  --o-visualization $data_dir/dada2_stats.qzv

Saved Visualization to: cancer_data/dada2_stats.qzv


In [3]:
Visualization.load(f'{data_dir}/dada2_table.qzv')

<visualization: Visualization uuid: b11a1cfc-5285-41da-b751-a9214fa287ed>

In [4]:
Visualization.load(f'{data_dir}/dada2_rep_set.qzv')

<visualization: Visualization uuid: 9fd7d0c4-bb2b-4568-b17f-ad834d0b765e>

In [5]:
Visualization.load(f'{data_dir}/dada2_stats.qzv')

<visualization: Visualization uuid: 91b7b7d6-760d-4e64-a14e-8dd45cec599d>

In [ ]:
! grep/cut...